In [4]:
from lyra import Client
from lyra.parser import parse_form
from lyra.transformer.transformer import transform_form
from IPython.display import display, Markdown

from fundar import json
from glob import glob
from tqdm.auto import tqdm

forms = glob('../data/*-form.json')

def display_markdown(md):
    display(Markdown(md))

with open('output.py', 'w') as f:
    f.write('# type: ignore\n\n')
    f.write('from lyra.common import OneOrMore\n\n')
    f.write('def download_data(*args, **kwargs): ...\n\n')
    for form in forms:
        ast = parse_form(json.load(form))
        collection_name = form.split('-form')[0].rsplit('/')[-1]
        display_str = [
            #  '```python\n'
            '# dataset: '
            , collection_name
            , f'\ndef download_{collection_name.replace("-", "_")}('
        ]

        transformed_ast = transform_form(ast)
        display_str.append(', '.join(str(x) for x in transformed_ast))
                        
        display_str.append('):')

        for p in transformed_ast:
            name = p.name
            valid_values = p.valid_values

            if not valid_values:
                continue

            valid_values = repr(valid_values)

            display_str += [
                # _valid_values = {valid_values}
                f'\n\t{name}_valid_values = {valid_values}'
                # assert {name} in {name}_valid_values
                , f'\n\tassert {name} in {name}_valid_values\n'
            ]


        display_str.append(
            "\n\treturn download_data(" + ', '.join(str(x.name) for x in transformed_ast) + ")\n"
        )

        display_str.append('\n\n')
        #display_str.append('\n```')

        f.write(''.join(display_str))